In [3]:
import scipy.io as sio
import skvideo.io
from IPython.display import Image as showimage
from PIL import Image
import numpy as np
import PXIO
import os, sys
import os.path
import random
import math

#########################################################################################################################
# Load videos & groundtruths
raw_info = sio.loadmat('raw_info.mat')['raw_info']
ordered_indices = sio.loadmat('ordered_indices.mat')['ordered_indices']
names = sio.loadmat('names.mat')['names']
utter_ids = sio.loadmat('utter_ids.mat')['utter_ids']
miss_ids = sio.loadmat('miss_ids.mat')['miss_ids']
indices_start_u = sio.loadmat('indices_start_u.mat')['indices_start_u']
indices_end_u = sio.loadmat('indices_end_u.mat')['indices_end_u']
pi = math.pi

#########################################################################################################################
# Step 0
# all_list = 'all_test_score_seq.txt'
# all_list = 'all_test_score_seq_103.txt' #10-24
# all_list = 'all_test_score_static_95.txt'
# subfolder = str(11)
# subfolder = 'gt'
subfolder = '3_lm_p9'
all_list = 'all_test_score_seq_' + subfolder + '.txt'
frame_list = [line.rstrip('\n') for line in open(all_list)]
frame_num = len(frame_list)
print(frame_num)

#########################################################################################################################
# Step 1
u_id = []
f_id = []
s_id = []
for i in range(0, frame_num):
    tmp = frame_list[i]
    token = tmp.split(' ')
    u_id.append(int(token[0])) # utterance
    f_id.append(int(token[1])) # frame
    s_id.append(float(token[2])) # score

#########################################################################################################################
# Step 2
id_num = u_id[0]
start = [0]
end = []
for i in range(0, len(u_id)):
    if u_id[i] != id_num:
        id_num = u_id[i]
        end.append(i-1)
        start.append(i)
end.append(len(u_id)-1)
start_x = start
end_x = end
#########################################################################################################################
# Step 3
def interp_(f_id, va, vp, vy, vr, start_j, end_j, start_idx, end_idx):
    f_num = end_j - start_j - 1
    diff_va = va[end_idx] - va[start_idx]
    diff_vp = vp[end_idx] - vp[start_idx]
    diff_vy = vy[end_idx] - vy[start_idx]
    diff_vr = vr[end_idx] - vr[start_idx]
    unit_va = diff_va / (f_num + 1)

    unit_vp = diff_vp / (f_num + 1)
    unit_vy = diff_vy / (f_num + 1)
    unit_vr = diff_vr / (f_num + 1)
    va_end = va[end_idx]
    vp_end = vp[end_idx]
    vy_end = vy[end_idx]
    vr_end = vr[end_idx]
    for i in range(1, f_num+1):
        va_ = va_end - i * unit_va
        vp_ = vp_end - i * unit_vp
        vy_ = vy_end - i * unit_vy
        vr_ = vr_end - i * unit_vr
        f_id=np.insert(f_id, end_idx, end_j-i)
        va=np.insert(va, end_idx, va_)
        vp=np.insert(vp, end_idx, vp_)
        vy=np.insert(vy, end_idx, vy_)
        vr=np.insert(vr, end_idx, vr_)
    return f_id, va, vp, vy, vr

ea_path = './eye_aperture_raw/'
for k in range(0, 19):
    ea_list = ea_path + names[k] + '.txt'
    ea_list = [line.rstrip('\n') for line in open(ea_list)]
    ea_list_size = len(ea_list)
    f_ea_id = []
    ea_v = []
    p_v = []
    y_v = []
    r_v = []

    for i in range(0, ea_list_size):
        tmp = ea_list[i]
        token = tmp.split(', ')
        f_ea_id.append(token[0].replace('frameID: ', ''))
        ea_v.append(token[1].replace('eyeAperture: ', ''))
        p_v.append(token[2].replace('pitch: ', ''))
        y_v.append(token[3].replace('yaw: ', ''))
        r_v.append(token[4].replace('roll: ', ''))

    f_id_ = [int(m) for m in f_ea_id]
    f_id_ = np.asarray(f_id_)
    ea_v_ = [float(m) for m in ea_v]
    ea_v_ = np.asarray(ea_v_)
    p_v_ = [float(m) for m in p_v]
    p_v_ = np.asarray(p_v_)
    y_v_ = [float(m) for m in y_v]
    y_v_ = np.asarray(y_v_)
    r_v_ = [float(m) for m in r_v]
    r_v_ = np.asarray(r_v_)

    miss_f = []
    miss_idx = []
    for i in range(0, ea_list_size-1):
        if int(f_ea_id[i+1]) - int(f_ea_id[i]) < 6 and int(f_ea_id[i+1]) - int(f_ea_id[i]) > 1:
            miss_f.append((int(f_ea_id[i]), int(f_ea_id[i+1])))
            miss_idx.append((i, i+1))


    f_id_new = f_id_
    ea_v_new = ea_v_
    p_v_new = p_v_
    y_v_new = y_v_
    r_v_new = r_v_
    for i in range(len(miss_f)-1, -1, -1):
        f_id_new, ea_v_new, p_v_new, y_v_new, r_v_new = interp_(f_id_new, ea_v_new,  p_v_new, y_v_new, r_v_new, \
                                               miss_f[i][0], miss_f[i][1], miss_idx[i][0], miss_idx[i][1])


    start = [0]
    end = []
    for i in range(0, len(f_id_new)-1):
        if int(f_id_new[i+1]) - int(f_id_new[i]) > 6: 
            start.append(i+1)
            end.append(i)
    end.append(len(f_id_new)-1)
    start = np.asarray(start)
    end = np.asarray(end)
    

    output_new = []
    for i in range(0, len(f_id_new)):
        # for eye aperture, yaw, pitch, roll
        line_ = ''
        line_ = line_ + 'frameID: ' + str(f_id_new[i]) 
        line_ = line_ + ', eyeAperture: ' + str(ea_v_new[i])
        line_ = line_ + ', pitch: ' + str(p_v_new[i])
        line_ = line_ + ', yaw: ' + str(y_v_new[i])
        line_ = line_ + ', roll: ' + str(r_v_new[i])
        output_new.append(line_)
        
    file_path = './eye_aperture/'
    file_name = names[k] + '.txt'
    PXIO.WriteLineToFile(file_path+file_name, output_new)
    
    
#########################################################################################################################
# Step 4
# id_list = [0,1,2,3,9,10,11,12,15,16,18]
# id_list = [4,5,6,7,8,13,14,17]
# for k in id_list:
for k in range(0, 19):
    # print(k)
    # for eye brows
    output_eb_lines = []
    utters = ordered_indices[0][k][0]
    for j , u_index in enumerate(utters):
#         real_u_index = np.where(utter_ids[0]==u_index)[0][0]
        real_u_index = u_index
#         print('j:', j, 'u_index:',real_u_index)
        for l in range(start_x[real_u_index], end_x[real_u_index]+1):
            line_eb = ''
            line_eb = line_eb + str(indices_start_u[0][k][0][j] + f_id[l] + 1) + ' ' + str(s_id[l])
            output_eb_lines.append(line_eb)
#             if l == start_x[real_u_index] and j ==84:
#                 print(line_eb)
#         if j == 84:
#             print(start_x[real_u_index], end_x[real_u_index])
#             print(line_eb)
    file_path = './eye_brow/'
    file_name = names[k] + '.txt'
    PXIO.WriteLineToFile(file_path+file_name, output_eb_lines)

    # for eye apertures
    eb_path = './eye_brow/' + names[k] + '.txt'
    eb_frame_list = [line.rstrip('\n') for line in open(eb_path)]
    eb_frame_num = len(eb_frame_list)
    eb_frames = []
    for i in range(0, eb_frame_num):
        eb_frames.append(int(eb_frame_list[i].split(' ')[0]))
    
    ea_raw_path = './eye_aperture/' + names[k] + '.txt'
    ea_frame_list = [line.rstrip('\n') for line in open(ea_raw_path)]
    ea_frame_num = len(ea_frame_list)
    ea_frames = []
    for i in range(0, ea_frame_num):
        ea_frames.append(int(ea_frame_list[i].split(', ')[0].replace('frameID: ', '')))
        
    ea_frames=np.asarray(ea_frames)
    eb_frames=np.asarray(eb_frames)

    output_ea_lines = []
#     for i in range(0, ea_frame_num):
#         if int(ea_frame_list[i].split(', ')[0].replace('frameID: ', '')) in eb_frames:
#             output_ea_lines.append(ea_frame_list[i])
    for i in range(0, eb_frame_num):
        if eb_frames[i] in ea_frames:
            output_ea_lines.append(ea_frame_list[np.where(ea_frames==eb_frames[i])[0][0]])
        else:
            output_ea_lines.append('frameID: ' + str(eb_frames[i]) +', eyeAperture: 0, pitch: 0, yaw: 0, roll: 0')
            
    file_path = './eye_aperture/'
    file_name = names[k] + '.txt'
    PXIO.WriteLineToFile(file_path+file_name, output_ea_lines)

#########################################################################################################################
# Step 5
ea_path = './eye_aperture/'
eb_path = './eye_brow/'
# id_list = [0,1,2,3,9,10,11,12,15,16,18]
# id_list = [4,5,6,7,8,13,14,17]
for k in range(0, 19):
    ea_list = ea_path + names[k] + '.txt'
    eb_list = eb_path + names[k] + '.txt'
    ea_list = [line.rstrip('\n') for line in open(ea_list)]
    eb_list = [line.rstrip('\n') for line in open(eb_list)]
    ea_list_size = len(ea_list)
    eb_list_size = len(eb_list)
    print(ea_list_size, eb_list_size)
    f_ea_id = []
    f_eb_id = []
    ea_v = []
    eb_v = []
    p_v = []
    y_v = []
    r_v = []

    for i in range(0, ea_list_size):
        tmp = ea_list[i]
        token = tmp.split(', ')
        f_ea_id.append(token[0].replace('frameID: ', ''))
        ea_v.append(token[1].replace('eyeAperture: ', ''))
        p_v.append(token[2].replace('pitch: ', ''))
        y_v.append(token[3].replace('yaw: ', ''))
        r_v.append(token[4].replace('roll: ', ''))

    for i in range(0, eb_list_size):
        tmp = eb_list[i]
        token = tmp.split(' ')
        f_eb_id.append(token[0])
        eb_v.append(token[1])

    a_id = [int(m) for m in f_ea_id]
    b_id = [int(m) for m in f_eb_id]
    a_id_ = np.asarray(a_id)
    b_id_ = np.asarray(b_id)
    diff = a_id_.shape[0] - b_id_.shape[0]
    s=np.sum(diff)
    print(s)

    f_id_ = [int(m) for m in f_eb_id]  #ea eb
    f_id_ = np.asarray(f_id_)
    ea_v_ = [float(m) for m in ea_v]
    ea_v_ = np.asarray(ea_v_)
    eb_v_ = [float(m) for m in eb_v]
    eb_v_ = np.asarray(eb_v_)
    p_v_ = [float(m) for m in p_v]
    p_v_ = np.asarray(p_v_)
    y_v_ = [float(m) for m in y_v]
    y_v_ = np.asarray(y_v_)
    r_v_ = [float(m) for m in r_v]
    r_v_ = np.asarray(r_v_)

    f_id_new = f_id_
    ea_v_new = ea_v_
    eb_v_new = eb_v_
    p_v_new = p_v_
    y_v_new = y_v_
    r_v_new = r_v_

    start = [0]
    end = []
    n=0
    for i in range(0, len(f_id_new)-1):
        if int(f_id_new[i+1]) - int(f_id_new[i]) > 6: 
            n = n + 1
            #print('start-end:', i+1, i, f_id_new[i+1], f_id_new[i])
            start.append(i+1)
            end.append(i)
    end.append(len(f_id_new)-1)
    start = np.asarray(start)
    end = np.asarray(end)
    n
    
#     for i in range(0, len(start)):
#         ea_v_u = ea_v_new[start[i]:end[i]+1]
#         eyeaptureMin = np.minimum(np.min(ea_v_u), 1.2)
#         eyeaptureMax = np.maximum(np.max(ea_v_u), 3)
#         ea_v_u_after = np.divide(np.subtract(ea_v_u, eyeaptureMin), (eyeaptureMax - eyeaptureMin))
#         ea_v_new[start[i]:end[i]+1] = ea_v_u_after

    eyeaptureMin = np.minimum(np.min(ea_v_new), 1.2)
    eyeaptureMax = np.maximum(np.max(ea_v_new), 3)
    ea_v_new = np.divide(np.subtract(ea_v_new, eyeaptureMin), (eyeaptureMax - eyeaptureMin))    
    
    print('k', k)
#     yawMin = np.min(y_v_new)
#     print('yawMin:', yawMin)
#     yawMax = np.max(y_v_new)
#     print('yawMax:', yawMax)
#     pitchMin = np.min(p_v_new)
#     print('pitchMin:', pitchMin)
#     pitchMax = np.max(p_v_new)
#     print('pitchMax:', pitchMax)
#     rollMin = np.min(r_v_new)
#     print('rollMin:', rollMin)
#     rollMax = np.max(r_v_new)
#     print('rollMax:', rollMax)
    max_rad = 0.856025
    
    output_new = []
    for i in range(0, len(f_id_new)):
#         time = int(float(f_id_new[i])/30*1000)  
        time = round(float(f_id_new[i]) * 1000 * 1001 / 30000, 4) 
        # for eye brows
        line_eb = ''
        line_eb = line_eb + 'time: ' + str(time) + ', '
        line_eb = line_eb + 'field_id: 40001, field_name: "eye brows", '
        line_eb = line_eb + 'field_value: ' + str(round(eb_v_new[i], 4))
        output_new.append(line_eb)
        # for eye aperture
        line_ea = ''
        line_ea = line_ea + 'time: ' + str(time) + ', '
        line_ea = line_ea + 'field_id: 40002, field_name: "eye aperture", '
        line_ea = line_ea + 'field_value: ' + str(round(ea_v_new[i], 4))
        output_new.append(line_ea)
        # for yaw
        line_y = ''
        line_y = line_y + 'time: ' + str(time) + ', '
        line_y = line_y + 'field_id: 50001, field_name: "yaw", '
        line_y = line_y + 'field_value: ' + str(round(y_v_new[i]/max_rad, 4))
        output_new.append(line_y)
        # for pitch
        line_p = ''
        line_p = line_p + 'time: ' + str(time) + ', '
        line_p = line_p + 'field_id: 50002, field_name: "pitch", '
        line_p = line_p + 'field_value: ' + str(round(-1*p_v_new[i]/max_rad, 4))
        output_new.append(line_p)
        # for roll
        line_r = ''
        line_r = line_r + 'time: ' + str(time) + ', '
        line_r = line_r + 'field_id: 50003, field_name: "roll", '
        line_r = line_r + 'field_value: ' + str(round(r_v_new[i]/max_rad, 4))
        output_new.append(line_r)
        
    file_path = './eye_full/' + subfolder + '/'
    file_name = names[k] + '.txt'
    PXIO.WriteLineToFile(file_path+file_name, output_new)

    utterance_path = './mat_mm_dd/' + subfolder + '/' + names[k] + '/'
    if not os.path.exists(utterance_path):
        os.mkdir(utterance_path)
    for i in range(0, len(start)):
        f_id_u = f_id_new[start[i]:end[i]+1]
        eb_v_u = eb_v_new[start[i]:end[i]+1]
        ea_v_u = ea_v_new[start[i]:end[i]+1]
        p_v_u = p_v_new[start[i]:end[i]+1]
        y_v_u = y_v_new[start[i]:end[i]+1]
        r_v_u = r_v_new[start[i]:end[i]+1]
        sio.savemat(utterance_path + str(i+1) + '.mat', {'f_id': f_id_u, 'eb_v': eb_v_u, 'ea_v': ea_v_u, 'p_v': p_v_u, 'y_v': y_v_u, 'r_v': r_v_u})
        
print('------ The End -------')

76571
3454 3454
0
k 0
12196 12196
0
k 1
6471 6471
0
k 2
2735 2735
0
k 3
12933 12933
0
k 4
3631 3631
0
k 5
3657 3657
0
k 6
2944 2944
0
k 7
2147 2147
0
k 8
1879 1879
0
k 9
2036 2036
0
k 10
2807 2807
0
k 11
1101 1101
0
k 12
2250 2250
0
k 13
3959 3959
0
k 14
2075 2075
0
k 15
3801 3801
0
k 16
5512 5512
0
k 17
983 983
0
k 18
------ The End -------
